In [2]:
from src.llm_plan.environments.static.three_switches_room import ThreeSwitchesRoom

env = ThreeSwitchesRoom("./environments/static/ThreeSwitchesRoom.json")
env.plan  # Simple plan to solve the Two Agents Vault environment

[['AgentR.pddl', 'AgentG.pddl', 'AgentB.pddl'], ['orchestrator.pddl']]

In [3]:
# Create the plan
from src.llm_plan.planner import Planner
from src.llm_plan.llm import GPT_4o

model = GPT_4o()
planner = Planner(env, model)

In [4]:
# Plan
responses = planner.plan()

----- AgentR->pddl -----

----- System Prompt -----
	You are an expert with PDDL. Provide a domain and problem file.

----- Prompt -----
	Your name is Agent R. You are in an environment with the following public information:
['There is a locked door in the room.', 'There are three switches: red, green, and blue.', 'The door unlocks only when all three switches are pressed at the same time.']
You have the following knowledge:
['I am Agent R', 'I can press the red switch']
This is your goal:
Help open the door by pressing the red switch at the same time as the others.
Think step by step and provide a PDDL domain and problem to solve the task.
Enclose the domain between <domain></domain> and the problem between <problem></problem>.
----- Response -----
	<domain>
(define (domain unlock-door)
  (:requirements :strips)
  (:predicates 
    (door-locked)
    (switch-pressed ?color)
    (can-press ?agent ?color)
    (acting ?agent))

  (:action press-switch
    :parameters (?agent ?color)
    :

In [ ]:
# Extract the pddl problem and domain from the LLM response
from src.llm_plan.parser import PDDLParser

pddl_parser = PDDLParser()

final_plan = responses["pddl_orchestrator"]
domain, problem = pddl_parser.parse(final_plan, from_file=False)

In [ ]:
# Obtain the plan with the solver
import subprocess

from src.llm_plan.config import SOLVER_BINARY, SOLVER_ARGS

with open("./tmp/problem.pddl", "w") as f:
    f.write(problem)

with open("./tmp/domain.pddl", "w") as f:
    f.write(domain)
    
# Launch the solver
command = [
    SOLVER_BINARY,
    *SOLVER_ARGS,
    "./tmp/sas_plan",
    "./tmp/domain.pddl",
    "./tmp/problem.pddl",
]

with open("./tmp/logs.txt", "w") as logfile:
    subprocess.run(command, stdout=logfile, stderr=subprocess.STDOUT)